## Open notebook in: 
| Colab                                 | Gradient                                                                                                                                         |
|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nicolepcx/Transformers-in-Action/blob/main/CH03/ch03_text_summarization_fine_tuning.ipynb)                                              | [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/Nicolepcx/Transformers-in-Action/blob/main/CH03/ch03_text_summarization_fine_tuning.ipynb)|             

# Download repo

In [1]:
# Clone repo, if it's not already cloned, to be sure all runs on smoothly
# on Colab, Kaggle or Paperspace
import os

if not os.path.isdir('Transformers-in-Action'):
    !git clone https://github.com/Nicolepcx/Transformers-in-Action.git
else:
    print('Repository already exists. Skipping clone.')


current_path = %pwd
if '/Transformers-in-Action' in current_path:
    new_path = current_path + '/utils'
else:
    new_path = current_path + '/Transformers-in-Action/utils'
%cd $new_path


Cloning into 'Transformers-in-Action'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 157 (delta 0), reused 1 (delta 0), pack-reused 152
Receiving objects: 100% (157/157), 636.04 KiB | 1.42 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/content/Transformers-in-Action/utils


# About this notebook

In this notebook, we will fine-tune the Facebook/mBART-50 many-to-many model on the MLSum dataset. In the previous notebook, we explored and evaluated various text summarization models, including mBART-50, and now we will take the next step by adapting the mBART-50 model specifically for summarizing German news article data.

Fine-tuning a pre-trained model can be a challenging process, as it requires a good understanding of the dataset, model architecture, and the underlying task. The process often involves hyperparameter optimization to find the best combination of settings that yield optimal performance. However, in this notebook, we will stick to the recommended model settings to simplify the fine-tuning process.

In this notebook, you will:

1. Gain an understanding of the fine-tuning process, its challenges, and the importance of hyperparameter optimization.
2. Learn how to prepare the MLSum dataset for the fine-tuning process, including data preprocessing and splitting.
3. Explore the recommended settings for the Facebook/mBART-50 model and understand their significance in the fine-tuning process.
4. Implement the fine-tuning process using the recommended settings and observe how the model adapts to the news article data.
5. Evaluate the fine-tuned model on multiple samples from the BillSum dataset, considering ROUGE as a comparison.


By the end of this notebook, you will have a deeper understanding of the fine-tuning process. You will also gain practical experience in fine-tuning the Facebook/mBART-50 model on the MLSum dataset and evaluating its performance on news article data. 

# Install requirements

In [2]:
from requirements import *

In [3]:
install_required_packages_ch04()

Installing chapter 4 requirements...

✅ transformers == 4.26.1 installation completed successfully!

✅ datasets == 2.10.1 installation completed successfully!

✅ evaluate==0.4.0 installation completed successfully!

✅ rouge_score==0.1.2 installation completed successfully!

✅ sacrebleu installation completed successfully!

✅ pyarrow==9.0.0 installation completed successfully!

✅ sentencepiece installation completed successfully!



# Imports

In [4]:
from setup import *
from utils import *

In [5]:
useGPU()

Have fun with this chapter!🥳


# Allocate enough RAM

Let us try to get a __GPU__ with at least __15GB RAM__ for our notebook.

In [6]:
# crash colab to get more RAM -> uncomment to use
#!kill -9 -1

We can execute the following command `!free -h`  to see if we have enough RAM and `!nvidia-smi` to get more info about our GPU type we got assigned.
If the allocated GPU is too small, the above cell can be used to run the command to crash the notebook hoping to get a better GPU after the crash, since the GPU is randomly allocated.


In [7]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           83Gi       1.2Gi        77Gi       2.0Mi       4.3Gi        81Gi
Swap:            0B          0B          0B


In [8]:
!nvidia-smi

Wed Apr 12 15:48:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    44W / 400W |      3MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import torch

if torch.cuda.is_available():
    gpu_device = torch.device('cuda')
    gpu_info = torch.cuda.get_device_properties(gpu_device)
    gpu_memory = gpu_info.total_memory / 1e9  # Convert bytes to gigabytes
    print(f"GPU: {gpu_info.name}, Total Memory: {gpu_memory:.2f} GB")
else:
    print("No GPU detected.")


GPU: NVIDIA A100-SXM4-40GB, Total Memory: 42.48 GB


In [10]:

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb=32"


In [11]:
import datasets
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, DataCollatorForSeq2Seq, Trainer, TrainingArguments, EarlyStoppingCallback
import evaluate
from transformers import EarlyStoppingCallback, DataCollatorForSeq2Seq, set_seed
from evaluate import load as load_evaluate_metric
from transformers.trainer_utils import EvalPrediction
import numpy as np
import warnings
from transformers import logging
from transformers import EvalPrediction
from datasets import load_metric




In [12]:
set_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [13]:
# Load the MLSUM dataset (German version)
dataset = load_dataset("mlsum", "de")

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")


Generating train split:   0%|          | 0/220887 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11394 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10701 [00:00<?, ? examples/s]

Dataset mlsum downloaded and prepared to /root/.cache/huggingface/datasets/mlsum/de/1.0.0/1b2d4e3020a63e9423caeccd38a04b02b7472d5694a33f4b2ae5b09e11e1f4cb. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 220887
    })
    validation: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 11394
    })
    test: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 10701
    })
})

## Data Preprocessing and Subset Selection

In this code snippet, we define a preprocessing function and apply it to a subset of the BillSum dataset for both training and testing purposes. The primary objective of this code is to prepare the data for the fine-tuning process of the Facebook/BART-large-cnn model, which we will use for legislative text summarization.

1. `preprocess_function`: This function takes an example from the dataset, tokenizes the input text and the corresponding summary, and returns the input and target encodings in the required format. The input text is truncated or padded to a maximum length of 1024 tokens, while the summary is limited to 128 tokens.

2. `set_range`: We set a range of 420 to select a subset of the dataset for training and testing purposes. This allows us to reduce the computational resources needed for the fine-tuning process while still providing a representative sample of the dataset.

3. `subset_train` and `subset_test`: We select the specified range of examples from the "train" and "ca_test" splits of the dataset and apply the preprocess_function to each example using the `map()` function. This results in preprocessed training and testing subsets.

4. `set_format`: We set the format of both subset_train and subset_test to "torch", which allows us to work with PyTorch tensors directly. We also specify the columns to include in the format: "input_ids", "attention_mask", and "labels".

By applying this code, we efficiently preprocess and prepare the BillSum dataset for the fine-tuning process, ensuring that the data is compatible with the Facebook/BART-large-cnn model and facilitating an effective training and evaluation workflow.

In [15]:
def preprocess_function(example):
    input_encoding = tokenizer(example["text"], max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
    target_encoding = tokenizer(example["summary"], max_length=56, truncation=True, padding="max_length", return_tensors="pt")
    return {"input_ids": input_encoding["input_ids"][0], "attention_mask": input_encoding["attention_mask"][0], "labels": target_encoding["input_ids"][0]}

subset_train = dataset["train"].select(range(1200)).map(preprocess_function)
subset_test = dataset["test"].select(range(400)).map(preprocess_function)

subset_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
subset_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [16]:
subset_train

Dataset({
    features: ['text', 'summary', 'topic', 'url', 'title', 'date', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1200
})


## Computing ROUGE Scores for Model Evaluation


This code snippet defines a function called compute_rouge, which calculates the ROUGE score for the model's predictions during evaluation. The function takes an EvalPrediction object as input, which contains the model's generated predictions and the corresponding reference summaries (ground truth).

Here's a step-by-step explanation of the function:

1. `rouge = evaluate.load('rouge')`: We load the ROUGE metric from the Hugging Face datasets library using the load() function. This metric will be used to calculate the similarity between the model's predictions and the reference summaries.

2. `predictions = tokenizer.batch_decode(eval_prediction.predictions, skip_special_tokens=True)`: We decode the model's predictions stored in `eval_prediction`.predictions using the tokenizer's `batch_decode()` function. We set skip_special_tokens=True to remove any special tokens (e.g., padding or EOS tokens) from the decoded text.

3. `references = tokenizer.batch_decode(eval_prediction.label_ids, skip_special_tokens=True)`: Similarly, we decode the reference summaries (ground truth) stored in `eval_prediction.label_ids` using the tokenizer's `batch_decode()` function, while also removing any special tokens.

4. `return rouge.compute(predictions=predictions, references=references)`: We compute the ROUGE score using the `rouge.compute()` function with the decoded predictions and reference summaries as input. The function returns the calculated ROUGE scores, which serve as the evaluation metric for the model's performance.

We need this function for the Trainer class provided by the Hugging Face transformers library. The Trainer class requires a custom function to compute metrics during the evaluation phase. By defining compute_rouge, we can integrate the ROUGE metric into the training and evaluation workflow, allowing us to assess the model's performance in generating summaries.

In [17]:
def compute_rouge(eval_prediction: EvalPrediction):
    rouge = evaluate.load('rouge')
    predictions = tokenizer.batch_decode(eval_prediction.predictions, skip_special_tokens=True)
    references = tokenizer.batch_decode(eval_prediction.label_ids, skip_special_tokens=True)

    return rouge.compute(predictions=predictions, references=references)



## Setting Up Training Arguments and Initializing Data Collator


This code snippet sets up the training arguments and initializes a data collator for fine-tuning the text summarization model. The TrainingArguments class from the Hugging Face transformers library is used to define various hyperparameters and settings for the training process.

1. `output_dir`: Specifies the directory where the trained model and related files will be saved.
2. `num_train_epochs`: Sets the number of training epochs to 3, which is within the recommended range.
3. `learning_rate`: Sets the learning rate to 3e-5, which is the recommended value for this model.
4. `per_device_train_batch_size` and `per_device_eval_batch_size`: Set the batch size for training and evaluation to 8, as recommended.
5. `gradient_accumulation_steps`: Sets the number of steps to accumulate gradients before performing an optimizer step. The recommended value of 1 is used here.
6. `warmup_steps`: Specifies the number of warm-up steps for the learning rate scheduler. The recommended value of 500 is used.
7. `weight_decay`: Sets the weight decay for the optimizer to 0.01, as recommended.
8. `logging_dir, logging_steps`: Define the directory for logging and the number of steps between each logging.
9. `evaluation_strategy`, `eval_steps`: Set the evaluation strategy to 'steps' and the number of steps between each evaluation to 160.
10. `load_best_model_at_end`: Indicates whether to load the best model found during training at the end of the process.
11. `save_steps`: Specifies the number of steps between each checkpoint save.
12. `seed`: Sets the random seed for reproducibility.
13. `report_to`: Disables reporting to external platforms by setting it to 'none'.

Next, the `DataCollatorForSeq2Seq` class is used to initialize a data collator. This collator is responsible for preparing batches of data during training. It takes the tokenizer, model, padding setting, and maximum length as input. The data collator ensures that the input sequences are padded or truncated to the specified maximum length (1024 tokens) and organizes the data into the required format for training the seq2seq model.

In [18]:
new_path = current_path + ''
%cd $new_path

/content


In [19]:
%pwd

'/content'

In [20]:
!ls

sample_data  Transformers-in-Action


In [21]:
training_args = TrainingArguments(
    output_dir='./best_model',
    num_train_epochs=1,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=160,
    evaluation_strategy='steps',
    eval_steps=160,
    load_best_model_at_end=True,
    save_steps=160,
    save_strategy='steps',
    seed=42,
    report_to='none',
    fp16=True
)


# Initialize the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True, max_length=1024)


## Fine-Tuning the Model with Trainer and Early Stopping Callback

This code snippet initializes and trains a Trainer object from the Hugging Face transformers library, which simplifies the fine-tuning process for our text summarization model. The Trainer class provides a high-level interface for training, fine-tuning, and evaluating models, streamlining the overall process and reducing boilerplate code.

The Trainer object is initialized with the following parameters:

1. `model`: The pre-trained model to be fine-tuned.
2. `args`: The TrainingArguments object containing the hyperparameters and settings for the training process.
3. `train_dataset` and `eval_dataset`: The preprocessed training and evaluation subsets of the BillSum dataset.
4. `compute_metrics`: The custom metric function, compute_rouge, which calculates ROUGE scores during evaluation.
5. `data_collator`: The DataCollatorForSeq2Seq object responsible for preparing batches of data during training.
6. `callbacks`: A list of callbacks, including the EarlyStoppingCallback, which monitors training progress and stops training if there is no significant improvement in the evaluation metric within a specified number of evaluation steps.

The benefits of using the Trainer class include:

- Simplified training and evaluation loop, allowing for easy implementation and reduced development time.
- Built-in support for distributed training and mixed precision, improving training efficiency.
- Automatic handling of learning rate scheduling, optimization, and gradient accumulation.
- Integration with custom metric functions, such as compute_rouge, for model evaluation.
- The EarlyStoppingCallback is an optional feature that monitors the model's performance during training and stops the process early if there is no significant improvement within a specified number of evaluation steps. This callback helps prevent overfitting and can save time and computational resources by terminating training when the model's performance plateaus.

In this code snippet, the EarlyStoppingCallback is set to monitor the evaluation metric with a patience of 10 evaluation steps and a minimum improvement threshold of 0.001. If the metric does not improve by at least 0.001 within 10 evaluation steps, the training process will be terminated.

Finally, the trainer.train() function is called to start the fine-tuning process for the text summarization model using the specified parameters and settings.

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=subset_train,
    eval_dataset=subset_test,
    compute_metrics=compute_rouge,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(10, 0.001)]
)

trainer.train()
trainer.save_model("./best_model")


Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: topic, summary, text, title, url, date. If topic, summary, text, title, url, date are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1200
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 150
  Number of trainable parameters = 610879488
You're using a MBart50TokenizerFast t

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./best_model
Configuration saved in ./best_model/config.json
Configuration saved in ./best_model/generation_config.json
Model weights saved in ./best_model/pytorch_model.bin


In [23]:
# This line suppresses UserWarnings originating from the transformers module. 
# By setting the filter to ignore warnings, you can avoid having your notebook 
# cluttered with potentially unnecessary warning messages while running your code.
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

# This line sets the verbosity level of the logging messages to ERROR. By doing 
# this, only error messages will be displayed in the notebook, hiding other log 
# messages with lower severity levels such as DEBUG, INFO, and WARNING. 
# This helps to keep the notebook output focused on errors that may require 
# your attention.
logging.set_verbosity(logging.ERROR)


## Function for Generating Summaries with a Fine-Tuned Mode

Info: This code defines a function called `generate_summary`, which takes a fine-tuned model, tokenizer, and an input text as arguments. The function generates a summary for the given input text using the model and tokenizer.

1. The device variable is set to use a GPU if available, otherwise, it falls back to using the CPU.
2. The model is moved to the appropriate device using model.to(device).
3. The `max_length` variable is set to the tokenizer's model maximum length.
4. The input text is split into chunks based on the `max_length` to avoid exceeding the model's token limit.
5. An empty list called summaries is initialized to store the generated summaries for each chunk.
6. The function iterates over each chunk, encoding it using the tokenizer and moving the encoding to the appropriate device.
7. The `model.generate()` function is called with the input encoding, specifying parameters like `max_length`, `min_length`, length_penalty, and num_beams to control the generation process.
8. The generated summary is decoded using the tokenizer and appended to the summaries list.
9. The final summary is created by joining the summaries of each chunk using spaces and returned by the function.

This function can be used to generate summaries for long input texts with the fine-tuned model and tokenizer.

In [24]:
# Load the saved best model
best_model = MBartForConditionalGeneration.from_pretrained("./best_model")

# Modify the generate_summary function to use the best model
def generate_summary(model, tokenizer, input_text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    max_length = 1024
    chunks = [input_text[i:i + max_length] for i in range(0, len(input_text), max_length)]
 
    summaries = []
    for chunk in chunks:
        input_encoding = tokenizer(chunk, max_length=max_length, truncation=True, return_tensors="pt")
        input_encoding = input_encoding.to(device)

        summary_ids = model.generate(input_encoding["input_ids"], max_length=56, length_penalty=2.0, num_beams=4, forced_bos_token_id=tokenizer.lang_code_to_id["de_DE"])
        summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary_text)

    summary = " ".join(summaries)
    return summary


## Evaluating the Model on 10 Test Samples and Computing ROUGE Scores

This following code evaluates the fine-tuned model on 10 samples from the test dataset and computes the ROUGE scores for the generated summaries.

- `generated_summaries`: A list comprehension is used to call the generate_summary function for each of the first 10 samples in the ca_test dataset. This generates a list of summaries produced by the model for each input text.

- `reference_summaries`: A list comprehension is used to extract the first 10 reference summaries from the ca_test dataset. These reference summaries serve as the ground truth for comparison with the generated summaries.

- `rouge = evaluate.load('rouge')`: The ROUGE evaluation metric is loaded using the evaluate library.

- `rouge_scores`: The rouge.compute() method is called with the predictions (generated_summaries) and references (reference_summaries) as its arguments. This computes the ROUGE scores (ROUGE-1, ROUGE-2, ROUGE-L, and ROUGELsum) for the generated summaries compared to the reference summaries.

We evaluate the model on different variants of ROUGE, which consider different aspects of the summaries, such as n-grams and the longest common subsequence. ROUGE was introduced in the paper titled "[ROUGE: A Package for Automatic Evaluation of Summaries](https://aclanthology.org/W04-1013.pdf)" by Chin-Yew Lin in 2004

__Note__: It's good practice to evaluate your model on different metrics and also compare it to a state-of-the-art model using the same dataset, if available. 

In [25]:
# Evaluate the model on 10 samples
generated_summaries = [generate_summary(best_model, tokenizer, dataset["test"]["text"][i]) for i in range(10)]
reference_summaries = [dataset["test"]["summary"][i] for i in range(10)]

rouge = evaluate.load('rouge')
rouge_scores = rouge.compute(predictions=generated_summaries, references=reference_summaries)
print(rouge_scores)

{'rouge1': 0.3061672022923375, 'rouge2': 0.2115558253398946, 'rougeL':
0.26244732289133355, 'rougeLsum': 0.26802471632495345}


In [26]:
# Create data
pd.DataFrame.from_records(rouge_scores, index=["mBART"])

,rouge1,rouge2,rougeL,rougeLsum
mBART,0.306167,0.211556,0.262447,0.268025


In [27]:
 print(dataset["test"]["text"][0])

Die Blockade vor Lampedusa ist durchbrochen, durch ein letztes
Manöver, mitten in der Nacht: Um 1.30 Uhr hat Carola Rackete, die
Kapitänin der Sea Watch 3, ihr Schiff an die Mole gesteuert. Dabei gab
es auch einige gefährliche Momente, für die Rackete sich inzwischen
entschuldigt hat: Das Motorboot der Guardia di Finanza, der
italienischen Zoll- und Steuerpolizei, hatte versucht, das
Rettungsschiff abzudrängen und zu stoppen. Doch die junge Kapitänin
war fest entschlossen, dem fast zweiwöchigen Kräftemessen mit der
italienischen Regierung ein Ende zu setzen und ihre Passagiere, 40
Migranten, an Land zu bringen. Racketes NGO rechtfertigte die
Landungsoperation mit einem extremen Notstand an Bord. Die Kapitänin
habe keine andere Wahl gehabt. Es gab nicht mehr genug Wasser an Bord,
niemand habe mehr duschen können, überall hätten sich Abfallberge
getürmt. Drei Mal habe sie deswegen Alarm geschlagen, drei Mal hätten
ihr die Italiener nicht geantwortet. Die Sorge um die psychische
Verfassun

In [28]:
print(generated_summaries[0])

Die Blockade vor Lampedusa ist durchbrochen, durch ein letztes
Manöver, mitten in der Nacht: Um 1.30 Uhr hat Carola Rackete, die
Kapitänin der Sea Watch 3, ihr Schiff an die Mole gesteuert. Kaum
hatte das Schiff angelegt, ging die Polizei an Bord und führte Carola
Rackete ab. Italienische Nachrichtensender übertrugen die Szenen live.
Ird man ihr vorwerfen, sie habe Beihilfe zur illegalen Einwanderung
geleistet, als sie am 12. Juni die zunächst 53 in Seenot geratenen
Migranten vor der Küste Libyens aufgenommen hatte und trotz eines
Angebots des nord "Menschenleben zu retten ist eine humanitäre
Verpflichtung", twitterte der SPD-Politiker am Samstag. dass man davor
ihre Daten in Italien aufnimmt. Der Hintergrund: Werden sie in Italien
registriert, mit Fotos und Fingerabdrücken, gilt Italien als Erstland.


In [29]:
print(reference_summaries[0])

Carola Rackete ist auf Lampedusa festgenommen worden. Außenminister
Heiko Maas und sein Kollege Jean Asselborn solidarisierten sich mit
der Kapitänin der "Sea Watch 3". Andere kritisierten ihr
Anlegemanöver.
